<div id="container" style="position:relative;">
<div style="float:left"><h1> Big Data Workflow With Google Books Ngrams </h1></div>  
      <div style="clear:both;"></div> 
<div style="float:left"><h2> Owner: Asli Keser </h2></div>
</div>

In [4]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1700074639840_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1700074639840_0003,pyspark,idle,Link,Link,None,✔


In [5]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.list_packages()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ----------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
click                      8.1.3
docutils                   0.14
jmespath                   1.0.1
joblib                     1.2.0
lockfile                   0.11.0
lxml                       4.9.2
mysqlclient                1.4.2
nltk                       3.8
nose                       1.3.4
numpy                      1.20.0
pip                        20.2.2
py-dateutil                2.2
pystache                   0.5.4
python-daemon              2.2.3
python37-sagemaker-pyspark 1.4.2
pytz                       2022.7
PyYAML                     5.4.1
regex                      2021.11.10
setuptools                 28.8.0
simplejson                 3.2.0
six                        1.13.0
tqdm                       4.64.1
wheel                      0.29.0
windmill                   1.6


In [7]:
sc.install_pypi_package("pandas==1.0.5") 
sc.install_pypi_package("matplotlib==3.1.1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# read csv file
df = spark.read.csv('/home/hadoop/eng_1M_1gram/eng_1M_1gram.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df.head(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(token='inGermany', year='1927', frequency='2', pages='2', books='2'), Row(token='inGermany', year='1929', frequency='1', pages='1', books='1'), Row(token='inGermany', year='1930', frequency='1', pages='1', books='1'), Row(token='inGermany', year='1933', frequency='1', pages='1', books='1'), Row(token='inGermany', year='1934', frequency='1', pages='1', books='1')]

In [18]:
# the structure of the data source
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- token: string (nullable = true)
 |-- year: string (nullable = true)
 |-- frequency: string (nullable = true)
 |-- pages: string (nullable = true)
 |-- books: string (nullable = true)

In [19]:
# the column names 
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['token', 'year', 'frequency', 'pages', 'books']

In [48]:
# print out the shape of the df
print(f'There are {df.count()} rows and {len(df.columns)} columns in the df.')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

There are 261823225 rows and 5 columns in the df.

In [20]:
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+---------+-----+-----+
|    token|year|frequency|pages|books|
+---------+----+---------+-----+-----+
|inGermany|1927|        2|    2|    2|
|inGermany|1929|        1|    1|    1|
|inGermany|1930|        1|    1|    1|
|inGermany|1933|        1|    1|    1|
|inGermany|1934|        1|    1|    1|
|inGermany|1935|        1|    1|    1|
|inGermany|1938|        5|    5|    5|
|inGermany|1939|        1|    1|    1|
|inGermany|1940|        1|    1|    1|
|inGermany|1942|        2|    2|    2|
+---------+----+---------+-----+-----+
only showing top 10 rows

In [21]:
df.show(10, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------
 token     | inGermany 
 year      | 1927      
 frequency | 2         
 pages     | 2         
 books     | 2         
-RECORD 1--------------
 token     | inGermany 
 year      | 1929      
 frequency | 1         
 pages     | 1         
 books     | 1         
-RECORD 2--------------
 token     | inGermany 
 year      | 1930      
 frequency | 1         
 pages     | 1         
 books     | 1         
-RECORD 3--------------
 token     | inGermany 
 year      | 1933      
 frequency | 1         
 pages     | 1         
 books     | 1         
-RECORD 4--------------
 token     | inGermany 
 year      | 1934      
 frequency | 1         
 pages     | 1         
 books     | 1         
-RECORD 5--------------
 token     | inGermany 
 year      | 1935      
 frequency | 1         
 pages     | 1         
 books     | 1         
-RECORD 6--------------
 token     | inGermany 
 year      | 1938      
 frequency | 5         
 pages     | 5         
 books     | 5  

In [29]:
# register the dataframe as a view
df.createOrReplaceTempView("temp_view")
# makes available the dataframe "df" in an SQL query like the one below

# execute a SQL query filtering token == "data"
spark.sql('SELECT * FROM temp_view WHERE token == "data"').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+---------+-----+-----+
|token|year|frequency|pages|books|
+-----+----+---------+-----+-----+
| data|1584|       16|   14|    1|
| data|1614|        3|    2|    1|
| data|1627|        1|    1|    1|
| data|1631|       22|   18|    1|
| data|1637|        1|    1|    1|
| data|1638|        2|    2|    1|
| data|1640|        1|    1|    1|
| data|1642|        1|    1|    1|
| data|1644|        4|    4|    1|
| data|1647|        1|    1|    1|
+-----+----+---------+-----+-----+
only showing top 10 rows

In [30]:
df_filtered = spark.sql('SELECT * FROM temp_view WHERE token == "data"')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_filtered.show(10, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0---------
 token     | data 
 year      | 1584 
 frequency | 16   
 pages     | 14   
 books     | 1    
-RECORD 1---------
 token     | data 
 year      | 1614 
 frequency | 3    
 pages     | 2    
 books     | 1    
-RECORD 2---------
 token     | data 
 year      | 1627 
 frequency | 1    
 pages     | 1    
 books     | 1    
-RECORD 3---------
 token     | data 
 year      | 1631 
 frequency | 22   
 pages     | 18   
 books     | 1    
-RECORD 4---------
 token     | data 
 year      | 1637 
 frequency | 1    
 pages     | 1    
 books     | 1    
-RECORD 5---------
 token     | data 
 year      | 1638 
 frequency | 2    
 pages     | 2    
 books     | 1    
-RECORD 6---------
 token     | data 
 year      | 1640 
 frequency | 1    
 pages     | 1    
 books     | 1    
-RECORD 7---------
 token     | data 
 year      | 1642 
 frequency | 1    
 pages     | 1    
 books     | 1    
-RECORD 8---------
 token     | data 
 year      | 1644 
 frequency | 4    
 pages     |

In [49]:
# Print out the shape of the df
print(f'There are {df_filtered.count()} rows and {len(df_filtered.columns)} columns in the df.')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

There are 316 rows and 5 columns in the df.

In [39]:
df_filtered.write.csv("/home/hadoop/eng_1M_1gram/eng_1M_1gram_filtered.csv",header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…